# Query file
This file queries the constituents of the S& P at different times.

In [1]:
##########################################
# S&P 500 Index Constituents             #
# Qingyi (Freda) Song Drechsler          #
# Date: October 2020                     #
##########################################

import pandas as pd
import wrds
from collections import Counter

In [2]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

Enter your WRDS username [juliusgruber]: jgruber99
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret,b.retx,b.bid, b.ask, b.shrout
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [4]:
sp500

,permno,start,ending,date,ret,retx,bid,ask,shrout
0,77178,1999-07-22,2023-12-29,2000-01-31,-0.278921,-0.278921,126.93750,127.00000,658276.0
1,48485,1976-07-01,2006-03-31,2000-01-31,-0.139194,-0.139194,58.50000,59.00000,104726.0
2,19502,1979-10-11,2023-12-29,2000-01-31,-0.044872,-0.044872,27.62500,27.87500,1006244.0
3,28847,1986-12-18,2002-05-14,2000-01-31,-0.311891,-0.311891,21.87500,22.25000,71459.0
4,56573,1986-02-13,2023-12-29,2000-01-31,-0.134135,-0.134135,58.25000,58.87500,300112.0
...,...,...,...,...,...,...,...,...,...
144482,22111,1973-05-31,2023-12-29,2023-12-29,0.013449,0.013449,156.74001,156.75999,2407279.0
144483,44206,2009-03-17,2023-12-29,2023-12-29,0.050244,0.038882,61.70000,61.72000,349326.0
144484,23473,1997-12-18,2023-12-29,2023-12-29,0.013815,0.006518,103.45000,103.48000,156908.0
144485,77274,2004-07-01,2023-12-29,2023-12-29,0.067363,0.057572,81.01000,81.03000,1246042.0


In [5]:
# Add Other Descriptive Variables

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

In [6]:
mse.columns

Index(['comnam', 'ncusip', 'namedt', 'nameendt', 'permno', 'shrcd', 'exchcd',
       'hsiccd', 'ticker'],
      dtype='object')

In [7]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]
sp500_full

,permno,start,ending,date,ret,retx,bid,ask,shrout,comnam,ncusip,namedt,nameendt,shrcd,exchcd,hsiccd,ticker
0,77178,1999-07-22,2023-12-29,2000-01-31,-0.278921,-0.278921,126.93750,127.00000,658276.0,QUALCOMM INC,74752510,1991-12-13,2004-06-09,11,3,3663,QCOM
7,48485,1976-07-01,2006-03-31,2000-01-31,-0.139194,-0.139194,58.50000,59.00000,104726.0,JEFFERSON PILOT CORP,47507010,1969-06-09,2001-08-23,11,1,6311,JP
13,19502,1979-10-11,2023-12-29,2000-01-31,-0.044872,-0.044872,27.62500,27.87500,1006244.0,WALGREEN CO,93142210,1968-01-02,2002-01-01,11,1,5912,WAG
23,28847,1986-12-18,2002-05-14,2000-01-31,-0.311891,-0.311891,21.87500,22.25000,71459.0,U S AIRWAYS GROUP INC,91190510,1997-02-24,2001-08-23,11,1,4512,U
26,56573,1986-02-13,2023-12-29,2000-01-31,-0.134135,-0.134135,58.25000,58.87500,300112.0,ILLINOIS TOOL WORKS INC,45230810,1973-03-13,2001-02-28,11,1,3569,ITW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080359,22111,1973-05-31,2023-12-29,2023-12-29,0.013449,0.013449,156.74001,156.75999,2407279.0,JOHNSON & JOHNSON,47816010,2021-12-21,2023-12-29,11,1,3841,JNJ
1080372,44206,2009-03-17,2023-12-29,2023-12-29,0.050244,0.038882,61.70000,61.72000,349326.0,EVERSOURCE ENERGY,30040W10,2021-03-01,2023-12-29,11,1,4924,ES
1080378,23473,1997-12-18,2023-12-29,2023-12-29,0.013815,0.006518,103.45000,103.48000,156908.0,CINCINNATI FINANCIAL CORP,17206210,2021-03-31,2023-12-29,11,3,6330,CINF
1080383,77274,2004-07-01,2023-12-29,2023-12-29,0.067363,0.057572,81.01000,81.03000,1246042.0,GILEAD SCIENCES INC,37555810,2020-07-28,2023-12-29,11,3,2830,GILD


In [8]:
sp500_full.to_csv('data/SP500_constituents.csv')